In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


In [10]:
# To create Test , Train and Validation DataGenerator

def preprocess_data(train_dir, val_dir, test_dir, target_size=(224, 224), batch_size=32):
    train_datagen = ImageDataGenerator()
    
    val_datagen = ImageDataGenerator()
    test_datagen = ImageDataGenerator()
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )
    
    return train_generator, validation_generator, test_generator

In [11]:
train_dir = './dataset/train'
val_dir = './dataset/validation'
test_dir = './dataset/test'
train_gen, val_gen, test_gen = preprocess_data(train_dir, val_dir, test_dir)

Found 36605 images belonging to 21 classes.
Found 3150 images belonging to 21 classes.
Found 3150 images belonging to 21 classes.


In [12]:
from model import model_transfer_learning_restnet_50

In [13]:
num_classes = len(train_gen.class_indices)  # Number of classes

In [14]:
model = model_transfer_learning_restnet_50(num_classes)

In [15]:
model.summary()

Model: "sound_classifier_restnet_50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, None, None, 3)     │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, None, None, 3)     │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, None, None, 64)    │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, None, None, 64)    │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, None, None, 64)    │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, None, None, 64)    │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, None, None, 64)    │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, None, None, 64)    │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, None, None, 64)    │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, None, None, 64)    │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, None, None, 64)    │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, None, None, 64)    │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, None, None, 64)    │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, None, None, 256)   │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, None, None, 256)   │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 25,707,413 (98.07 MB)

 Trainable params: 2,119,701 (8.09 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [25]:

early_stopping = EarlyStopping(monitor = 'val_accuracy',
                              patience = 5, mode = 'max',
                              restore_best_weights = True)

checkpoint = ModelCheckpoint('best_model.keras',
                            monitor = 'val_accuracy',
                            save_best_only = True)
model.fit(train_gen, validation_data=val_gen, epochs=10)
       

Epoch 1/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4080 - loss: 1.9066

C:\Users\DC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1144/1144 ━━━━━━━━━━━━━━━━━━━━ 1597s 1s/step - accuracy: 0.4080 - loss: 1.9065 - val_accuracy: 0.3524 - val_loss: 1.9077
Epoch 2/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 1602s 1s/step - accuracy: 0.4904 - loss: 1.4994 - val_accuracy: 0.3698 - val_loss: 1.8158
Epoch 3/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 2144s 2s/step - accuracy: 0.5116 - loss: 1.4047 - val_accuracy: 0.3810 - val_loss: 1.7734
Epoch 4/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 2202s 2s/step - accuracy: 0.5214 - loss: 1.3714 - val_accuracy: 0.3759 - val_loss: 1.7834
Epoch 5/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 2121s 2s/step - accuracy: 0.5224 - loss: 1.3572 - val_accuracy: 0.3784 - val_loss: 1.7650
Epoch 6/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 1781s 2s/step - accuracy: 0.5329 - loss: 1.3217 - val_accuracy: 0.3797 - val_loss: 1.7632
Epoch 7/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 1715s 1s/step - accuracy: 0.5337 - loss: 1.3164 - val_accuracy: 0.3949 - val_loss: 1.7377
Epoch 8/10
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 1594s 1s/step - accuracy: 0.5381 - loss: 1.30

In [26]:
model.save("restnet_50.keras")